# Chapter 15: Processing Sequences Using RNNs and CNNs

Bab ini membahas **Recurrent Neural Networks (RNN)**, kelas model yang dirancang khusus untuk menangani data berurutan. Tidak seperti jaringan saraf konvensional yang menganggap semua input saling independen, RNN memiliki "ingatan" karena mereka mempertahankan *state* internal berdasarkan apa yang telah mereka lihat sebelumnya.

## Mengapa RNN?
RNN sangat berguna untuk:
- **Time Series Forecasting**: Memprediksi harga saham, cuaca, atau permintaan energi.
- **Natural Language Processing (NLP)**: Terjemahan mesin, analisis sentimen, dan chatbot.
- **Audio Analysis**: Pengenalan suara (speech-to-text).

## Daftar Isi:
1. **Konsep Dasar RNN**: Neuron Rekuren dan Memori.
2. **Pelatihan RNN**: Backpropagation Through Time (BPTT).
3. **Peramalan Time Series**: Memprediksi nilai masa depan.
4. **Menangani Urutan Panjang**: Masalah memori jangka pendek.
5. **Sel LSTM dan GRU**: Arsitektur untuk memori jangka panjang.
6. **CNN 1D untuk Urutan**: Alternatif RNN yang lebih cepat.
7. **WaveNet**: Arsitektur canggih untuk audio.

## 1. Bagaimana RNN Bekerja?

Pada setiap langkah waktu $t$, neuron rekuren menerima input $x_{(t)}$ serta outputnya sendiri dari langkah waktu sebelumnya $h_{(t-1)}$ (juga disebut *hidden state*). 

### Dimensi Input RNN
Lapisan RNN di Keras mengharapkan input 3D dengan bentuk: `[batch_size, time_steps, dimensionality]`.
- **batch_size**: Jumlah sampel per batch.
- **time_steps**: Panjang urutan (misal: 50 hari terakhir).
- **dimensionality**: Jumlah fitur di setiap langkah (misal: suhu, kelembaban).

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# Contoh RNN Sederhana
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)
])

print("Model RNN dengan 2 lapisan rekuren berhasil didefinisikan.")

## 2. Peramalan Time Series (Forecasting)

Sebelum menggunakan model kompleks, kita harus selalu memiliki **Baseline Metrics**:
1. **Naive Forecasting**: Memprediksi bahwa nilai besok akan sama dengan hari ini.
2. **Linear Regression**: Model dasar untuk melihat tren linear.

### Deep RNN vs Simple RNN
Dalam Deep RNN, kita menumpuk beberapa lapisan rekuren. Penting untuk mengatur `return_sequences=True` pada semua lapisan kecuali yang terakhir, agar lapisan berikutnya menerima urutan lengkap sebagai input.

In [ ]:
# Membangun Deep RNN untuk peramalan
model_deep = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10)) # Memprediksi 10 langkah ke depan sekaligus
])

model_deep.summary()

## 3. Masalah Short-Term Memory

RNN standar (SimpleRNN) menderita masalah gradien hilang (*vanishing gradients*), sehingga mereka melupakan informasi dari awal urutan yang panjang. Solusinya adalah menggunakan sel yang lebih canggih:

### LSTM (Long Short-Term Memory)
Memperkenalkan *cell state* (jalur memori jangka panjang) dan tiga gerbang (*gates*):
- **Forget Gate**: Memutuskan informasi apa yang dibuang.
- **Input Gate**: Memutuskan informasi baru apa yang disimpan.
- **Output Gate**: Memutuskan bagian memori mana yang dikeluarkan sebagai output.

### GRU (Gated Recurrent Unit)
Versi sederhana dari LSTM dengan parameter yang lebih sedikit namun performa yang seringkali setara.

In [ ]:
# Menggunakan LSTM
model_lstm = keras.models.Sequential([
    keras.layers.LSTM(50, return_sequences=True, input_shape=[None, 1]),
    keras.layers.LSTM(50),
    keras.layers.Dense(1)
])

print("Model LSTM siap digunakan untuk urutan panjang.")

## 4. CNN 1D untuk Memproses Urutan

Mengejutkannya, CNN 1D seringkali bekerja sangat baik untuk urutan dan jauh lebih cepat daripada RNN karena mereka bisa diproses secara paralel. CNN 1D meluncurkan filter di sepanjang sumbu waktu untuk mendeteksi pola lokal.

In [ ]:
# Menggunakan CNN 1D untuk Time Series
model_cnn = keras.models.Sequential([
    keras.layers.Conv1D(filters=20, kernel_size=4, strides=2, padding="valid", input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)
])

model_cnn.summary()

## 5. WaveNet

WaveNet menggunakan **Dilated Convolutions** (konvolusi melebar) untuk memperluas *receptive field* secara eksponensial tanpa meningkatkan jumlah parameter secara drastis. Ini memungkinkan model untuk melihat ribuan langkah waktu ke belakang, sangat efektif untuk audio resolusi tinggi.

## Rangkuman Praktis Bab 15

1. **RNN** adalah pilihan utama untuk data berurutan, namun **SimpleRNN** terbatas pada urutan pendek.
2. **LSTM dan GRU** wajib digunakan jika urutan Anda memiliki ketergantungan jangka panjang (long-term dependencies).
3. **CNN 1D** adalah alternatif yang sangat cepat dan seringkali kompetitif untuk peramalan time series.
4. Untuk peramalan, selalu bandingkan performa model Anda dengan **Naive Forecasting** agar Anda tahu apakah model benar-benar belajar pola atau hanya menebak nilai terakhir.
5. Gunakan `TimeDistributed` jika Anda ingin memprediksi nilai untuk setiap langkah waktu dalam urutan output.